#### Install Dependencies

In [ ]:
!sudo apt-get install jags
!pip install pyjags

# Bayesian Parameter Estimation for Factor Contributions

The following is a simplified template for using JAGS for analyses

In [ ]:
import pandas as pd
import numpy as np
import pyjags
import os

In [ ]:
DATA_PATH = 'content'
INPUT_FILE = 'ceda-results.csv'

### Read in data

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH,INPUT_FILE))

In [ ]:
# add any additional processing steps here

In [ ]:
df.head()

## Model Definition

### Temporal Change Point Model

The following is an example JAGS Script for the purposes of identifying change points in convergence-entropy throughout the course of a conversation.

In [ ]:
model = """
model{
    nxsigma ~ dunif(.001, 100)
    nysigma ~ dunif(.001, 100)
    
    nxmu ~ dunif(-1000, 1000)
    nymu ~ dunif(-1000, 1000)
    
    nx_beta ~ dnorm(nxmu, nxsigma)
    ny_beta ~ dnorm(nymu, nysigma)
    
    phi ~ dunif(.001, 100)
    
    change_point[1] ~ dunif(0,.000001)
    
    for (i in 1:n_cat){
        change_point[i+1] ~ dunif(ifelse(i>1, change_point[i], 0), t_max)
        change_rate[i] ~ dnorm(0, .001)
    }
    
    for (j in 1:ROWS){
        
        for (i in 1:n_cat){
            theta[j,i] <- ifelse(time[j] >= change_point[i], change_rate[i], 0)
        } 
        
        mu_row[j] <- sum(theta[j,]) + (nx_beta * nx[j]) + (ny_beta * ny[j])
        H[j] ~ dnorm(mu_row[j], phi)
    }
}
"""

Make sure that you include inputs for each variable that you are feeding to the model from the data, as indicated in the script you wrote.

In [ ]:
data = {
    'nx': df['nx'].values,
    'ny': df['ny'].values,
    'time': df['time_delta'].values,
    't_max': df['time_delta'].max(),
    'n_cat': 10, # the maximum number of possible change points to search for
    
    'H': df['Hxy'].values,
    # 'TEMP': 1,  # how far off from the correct value of H you are allowing the model to estimate.
                #  higher values will increase model performance, but decrease model certainty
                #  for any specific parameter values.
    'ROWS': len(df)
}

In [ ]:
n_samples = 3000
chains = 1
warm_up = 1000

jags_model = pyjags.Model(
    model,
    data=data,
    chains=chains,
    adapt=warm_up, 
    progress_bar=True
)

samples = jags_model.sample(n_samples,vars=['change_rate', 'change_point', 'nx_beta', 'ny_beta'])

saving outputs

In [ ]:
for param,vals in samples.items():
    np.save(
        os.path.join(DATA_PATH, param+'.npy'), 
        vals
    )